In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
from gensim.models import KeyedVectors, Word2Vec

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.manifold import TSNE
from string import punctuation

In [4]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
#!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

## Запилим пословный машинный перевод!

In [5]:
!wget -O ukr_rus.train.txt -qq --no-check-certificate "https://drive.google.com/uc?export=download&id=1vAK0SWXUqei4zTimMvIhH3ufGPsbnC_O"
!wget -O ukr_rus.test.txt -qq --no-check-certificate "https://drive.google.com/uc?export=download&id=1W9R2F8OeKHXruo2sicZ6FgBJUTJc8Us_"
!wget -O fairy_tale.txt -qq --no-check-certificate "https://drive.google.com/uc?export=download&id=1sq8zSroFeg_afw-60OmY8RATdu_T1tej"

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

downloaded = drive.CreateFile({'id': '1d7OXuil646jUeDS1JNhP9XWlZogv6rbu'})
downloaded.GetContentFile('cc.ru.300.vec.zip')

downloaded = drive.CreateFile({'id': '1yAqwqgUHtMSfGS99WLGe5unSCyIXfIxi'})
downloaded.GetContentFile('cc.uk.300.vec.zip')

!unzip cc.ru.300.vec.zip
!unzip cc.uk.300.vec.zip

Archive:  cc.ru.300.vec.zip
replace cc.ru.300.vec? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
Archive:  cc.uk.300.vec.zip
replace cc.uk.300.vec? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


Напишем простенькую реализацию модели машинного перевода.

Идея основана на статье [Word Translation Without Parallel Data](https://arxiv.org/pdf/1710.04087.pdf). У авторов в репозитории еще много интересного: [https://github.com/facebookresearch/MUSE](https://github.com/facebookresearch/MUSE).

А мы будем переводить с украинского на русский.

![](https://raw.githubusercontent.com/yandexdataschool/nlp_course/master/resources/blue_cat_blue_whale.png)   
*синій кіт* vs. *синій кит*

In [6]:
ru_emb = KeyedVectors.load_word2vec_format("cc.ru.300.vec")
uk_emb = KeyedVectors.load_word2vec_format("cc.uk.300.vec")

Посмотрим на пару серпень-август (являющихся переводом)

In [7]:
ru_emb.most_similar([ru_emb["август"]])

[('август', 1.0),
 ('июль', 0.9383153915405273),
 ('сентябрь', 0.9240028858184814),
 ('июнь', 0.9222575426101685),
 ('октябрь', 0.9095538854598999),
 ('ноябрь', 0.8930036425590515),
 ('апрель', 0.8729087114334106),
 ('декабрь', 0.8652557730674744),
 ('март', 0.8545796275138855),
 ('февраль', 0.8401416540145874)]

In [8]:
uk_emb.most_similar([uk_emb["серпень"]])

[('серпень', 0.9999999403953552),
 ('липень', 0.9096440076828003),
 ('вересень', 0.901697039604187),
 ('червень', 0.8992519378662109),
 ('жовтень', 0.8810408711433411),
 ('листопад', 0.8787633776664734),
 ('квітень', 0.8592804670333862),
 ('грудень', 0.8586863279342651),
 ('травень', 0.8408110737800598),
 ('лютий', 0.8256431818008423)]

In [9]:
ru_emb.most_similar([uk_emb["серпень"]])

[('Недопустимость', 0.24435284733772278),
 ('конструктивность', 0.23293080925941467),
 ('офор', 0.23256804049015045),
 ('deteydlya', 0.23031717538833618),
 ('пресечении', 0.22632381319999695),
 ('одностороннего', 0.22608885169029236),
 ('подход', 0.2230587601661682),
 ('иболее', 0.22003726661205292),
 ('2015Александр', 0.21872764825820923),
 ('конструктивен', 0.21796566247940063)]

In [10]:
def load_word_pairs(filename):
    uk_ru_pairs = []
    uk_vectors = []
    ru_vectors = []
    with open(filename, "r", encoding='utf8') as inpf:
        for line in inpf:
            uk, ru = line.rstrip().split("\t")
            if uk not in uk_emb or ru not in ru_emb:
                continue
            uk_ru_pairs.append((uk, ru))
            uk_vectors.append(uk_emb[uk])
            ru_vectors.append(ru_emb[ru])
    return uk_ru_pairs, np.array(uk_vectors), np.array(ru_vectors)


uk_ru_train, X_train, Y_train = load_word_pairs("ukr_rus.train.txt")
uk_ru_test, X_test, Y_test = load_word_pairs("ukr_rus.test.txt")

### Учим маппинг из одного пространства эмбеддингов в другое

У нас есть пары слов, соответствующих друг другу, и их эмбеддинги. Найдем преобразование из одного пространства в другое, чтобы приблизить известные нам слова:

$$W^*= \arg\min_W ||WX - Y||_F, \text{где} ||*||_F - \text{норма Фробениуса}$$

Эта функция очень похожа на линейную регрессию (без биаса).

**Задание** Реализуйте её - воспользуйтесь `LinearRegression` из sklearn с `fit_intercept=False`:

In [11]:
from sklearn.linear_model import LinearRegression

In [12]:
mapping = LinearRegression(fit_intercept=False).fit(X_train, Y_train)

Проверим, куда перейдет `серпень`:

In [13]:
august = mapping.predict(uk_emb["серпень"].reshape(1, -1))
ru_emb.most_similar(august)

[('апрель', 0.8541285991668701),
 ('июнь', 0.8411202430725098),
 ('март', 0.839699387550354),
 ('сентябрь', 0.835986852645874),
 ('февраль', 0.8329297304153442),
 ('октябрь', 0.8311845660209656),
 ('ноябрь', 0.8278923034667969),
 ('июль', 0.8234529495239258),
 ('август', 0.8120501041412354),
 ('декабрь', 0.803900420665741)]

Должно получиться, что в топе содержатся разные месяцы, но август не первый.

Будем мерять percision top-k с k = 1, 5, 10.

**Задание** Реализуйте следующую функцию:

In [14]:
def precision(pairs, mapped_vectors, mapping_word2vec,topn=1):
    """
    :args:
        pairs = list of right word pairs [(uk_word_0, ru_word_0), ...]
        mapped_vectors = list of embeddings after mapping from source embedding space to destination embedding space
        topn = the number of nearest neighbours in destination embedding space to choose from
    :returns:
        precision_val, float number, total number of words for those we can find right translation at top K.
    """
    assert len(pairs) == len(mapped_vectors)

    num_matches = 0
    for i, (_, ru) in enumerate(pairs):
      similar_words = mapping_word2vec.most_similar(mapped_vectors[i].reshape(1,-1), topn=topn)
      if ru in {el[0] for el in similar_words}:
        num_matches += 1
    precision_val = num_matches / len(pairs)
    return precision_val

In [15]:
assert precision([("серпень", "август")], august, ru_emb, topn=5) == 0.0
assert precision([("серпень", "август")], august, ru_emb, topn=9) == 1.0
assert precision([("серпень", "август")], august, ru_emb, topn=10) == 1.0

In [16]:
assert precision(uk_ru_test, X_test, ru_emb) == 0.0
assert precision(uk_ru_test, Y_test, ru_emb) == 1.0

In [17]:
precision_top1 = precision(uk_ru_test, mapping.predict(X_test), ru_emb, 1)
precision_top5 = precision(uk_ru_test, mapping.predict(X_test), ru_emb, 5)

assert precision_top1 >= 0.635
assert precision_top5 >= 0.813

### Улучшаем маппинг

Можно показать, что маппинг лучше строить ортогональным:
$$W^*= \arg\min_W ||WX - Y||_F \text{, где: } W^TW = I$$

Искать его можно через SVD:
$$X^TY=U\Sigma V^T\text{, singular value decompostion}$$

$$W^*=UV^T$$

**Задание** Реализуйте эту функцию.

In [18]:
def learn_transform(X_train, Y_train):
    """ 
    :returns: W* : float matrix[emb_dim x emb_dim] as defined in formulae above
    """
    U, D, VT = np.linalg.svd(X_train.T.dot(Y_train))
    return U.dot(VT)

In [19]:
W = learn_transform(X_train, Y_train)

In [20]:
ru_emb.most_similar([np.matmul(uk_emb["серпень"], W)])

[('апрель', 0.8237907886505127),
 ('сентябрь', 0.8049713373184204),
 ('март', 0.8025654554367065),
 ('июнь', 0.8021842241287231),
 ('октябрь', 0.8001736402511597),
 ('ноябрь', 0.7934483885765076),
 ('февраль', 0.7914121150970459),
 ('июль', 0.7908109426498413),
 ('август', 0.7891016602516174),
 ('декабрь', 0.7686373591423035)]

In [21]:
assert precision(uk_ru_test, np.matmul(X_test, W), ru_emb) >= 0.653
assert precision(uk_ru_test, np.matmul(X_test, W), ru_emb, topn=5) >= 0.824

### Пишем переводчик

Реализуем простой пословный переводчик - для каждого слова будем искать его ближайшего соседа в общем пространстве эмбеддингов. Если слова нет в эмбеддингах - просто копируем его.

In [22]:
with open("fairy_tale.txt", "r") as f:
    uk_sentences = [line.rstrip().lower() for line in f]

In [23]:
def translate(sentence, original, target, mapping, my_print=False):
    """
    :args:
        sentence - sentence in Ukrainian (str)
    :returns:
        translation - sentence in Russian (str)

    * find ukrainian embedding for each word in sentence
    * transform ukrainian embedding vector
    * find nearest russian word and replace
    """
    sentence_vector = [original[word] if (word in original.vocab and word.isalpha()) else word for word in sentence.split()]
    transform_vector = [mapping.predict(vec.reshape(1, -1)) if type(vec) is np.ndarray  else vec for vec in sentence_vector]
    if my_print == True:
      print(' '.join([target.most_similar(vec, topn=1)[0][0] if type(vec) is np.ndarray else '\033[34m'+vec+'\033[0m' for vec in transform_vector])+'\n')
    else:
      return ' '.join([target.most_similar(vec, topn=1)[0][0] if type(vec) is np.ndarray else vec for vec in transform_vector])

In [24]:
assert translate(".", uk_emb, ru_emb, mapping) == "."
assert translate("1 , 3", uk_emb, ru_emb, mapping) == "1 , 3"
assert translate("кіт зловив мишу", uk_emb, ru_emb, mapping) == "кот поймал мышь"

Переведем сказку с украинского языка на русский, слова без перевода оставим как есть и выделим синим цветом.

In [25]:
for sentence in uk_sentences:
    print(f'src: {sentence}\ndst:', end=' ')
    translate(sentence, uk_emb, ru_emb, mapping, my_print=True)

src: лисичка - сестричка і вовк - панібрат
dst: лисичка - девочка и волк - панібрат

src: як була собі лисичка , да й пішла раз до однії баби добувать огню ; ввійшла у хату да й каже : " добрий день тобі , бабусю !
dst: как была себе лисичка , ой и пошла раз от однії бабы добувать огнь ; лечь во избу ой и говорит : " хороший день тебе , бабуля !

src: дай мені огня " .
dst: дай мне из " .

src: а баба тільки що вийняла із печі пирожок із маком , солодкий , да й положила , щоб він прохолов ; а лисичка се і підгледала , да тілько що баба нахилилась у піч , щоб достать огня , то лисичка зараз ухватила пирожок да і драла з хати , да , біжучи , весь мак із його виїла , а туда сміття наклала .
dst: а баба только что вынуть со печи пирожок со маком , сладкий , ой и легла , чтобы он прохолов ; а лисичка коль и підгледала , ой притом что баба наклониться во печь , чтобы достать из , то лисичка сейчас ухватила пирожок ой и сожрать со избы , ой , бежать , весь мак со его виїла , а туда мусор нало

# Дополнительные материалы

## Почитать
### База:  
[On word embeddings - Part 1, Sebastian Ruder](http://ruder.io/word-embeddings-1/)  
[Deep Learning, NLP, and Representations, Christopher Olah](http://colah.github.io/posts/2014-07-NLP-RNNs-Representations/)  

### Как кластеризовать смыслы многозначных слов:  
[Making Sense of Word Embeddings (2016), Pelevina et al](http://anthology.aclweb.org/W16-1620)    

### Как оценивать эмбеддинги
[Evaluation methods for unsupervised word embeddings (2015), T. Schnabel](http://www.aclweb.org/anthology/D15-1036)  
[Intrinsic Evaluation of Word Vectors Fails to Predict Extrinsic Performance (2016), B. Chiu](https://www.aclweb.org/anthology/W/W16/W16-2501.pdf)  
[Problems With Evaluation of Word Embeddings Using Word Similarity Tasks (2016), M. Faruqui](https://arxiv.org/pdf/1605.02276.pdf)  
[Improving Reliability of Word Similarity Evaluation by Redesigning Annotation Task and Performance Measure (2016), Oded Avraham, Yoav Goldberg](https://arxiv.org/pdf/1611.03641.pdf)  
[Evaluating Word Embeddings Using a Representative Suite of Practical Tasks (2016), N. Nayak](https://cs.stanford.edu/~angeli/papers/2016-acl-veceval.pdf)  


## Посмотреть
[Word Vector Representations: word2vec, Lecture 2, cs224n](https://www.youtube.com/watch?v=ERibwqs9p38)